<a href="https://colab.research.google.com/github/Dareen57/DVE-Project/blob/main/DVEproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import seaborn as sns  # only for quick static plots in the notebook (not required in Dash)
import warnings
warnings.filterwarnings("ignore")


# Reload raw crash and person datasets from NYC Open Data

In [ ]:

crashes_url = 'https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv?accessType=download'
persons_url = 'https://data.cityofnewyork.us/api/views/f55k-p6yu/rows.csv?accessType=download'

df_crashes = pd.read_csv(crashes_url, low_memory=False)
df_persons = pd.read_csv(persons_url, low_memory=False)

print("Crashes shape:", df_crashes.shape)
print("Persons shape:", df_persons.shape)


# See info about datasets

In [ ]:
df_crashes.head(3).T

df_crashes.info()
df_persons.info()


# See missing values

In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(df_crashes.isnull(), cbar=False)
plt.title("Missing Data Heatmap (df_crashes)")
plt.show()


In [ ]:
def missing_summary(df, top_n=20):
    miss = df.isna().sum().sort_values(ascending=False)
    pct = (miss / len(df) * 100).round(2)
    return pd.DataFrame({"missing_count": miss, "missing_pct": pct}).head(top_n)

missing_summary(df_crashes)


# count duplicates and see how many unique collision IDs are there

In [ ]:
dup_count = df_crashes.duplicated().sum()
print("Exact duplicate rows in crashes:", dup_count)


if "COLLISION_ID" in df_crashes.columns:
    print("Unique COLLISION_IDs:", df_crashes["COLLISION_ID"].nunique())
else:
    print("No COLLISION_ID in crashes (older schema maybe).")


contributing factor frequency

In [ ]:

import plotly.express as px

df_factor = (
    df_crashes["CONTRIBUTING FACTOR VEHICLE 1"]
    .value_counts()
    .head(15)
    .reset_index()
)

df_factor.columns = ["Factor", "Count"]

px.bar(
    df_factor,
    x="Factor",
    y="Count",
    title="Top Contributing Factors in Crashes"
).show()


# Parse data and time, combine it, and applying it to the crash dataset to use it later

In [ ]:
# Cell 6: Parse date/time safely (using your actual crash column names)
def parse_datetime(df, date_col="CRASH DATE", time_col="CRASH TIME"):
    # Clean whitespace and ensure string type
    df[date_col] = df[date_col].astype(str).str.strip()
    df[time_col] = df[time_col].astype(str).str.strip()

    # Combine date and time into one datetime column
    df["CRASH_DATETIME"] = pd.to_datetime(
        df[date_col] + " " + df[time_col],
        errors="coerce",
        infer_datetime_format=True
    )

    # Extract useful components
    df["CRASH_YEAR"] = df["CRASH_DATETIME"].dt.year
    df["CRASH_MONTH"] = df["CRASH_DATETIME"].dt.month
    df["CRASH_DAY"] = df["CRASH_DATETIME"].dt.day
    df["CRASH_WEEKDAY"] = df["CRASH_DATETIME"].dt.day_name()
    df["CRASH_HOUR"] = df["CRASH_DATETIME"].dt.hour

    return df

# Apply to the crash dataset
df_crashes = parse_datetime(df_crashes)

# Preview key columns
df_crashes[["CRASH DATE", "CRASH TIME", "CRASH_DATETIME", "CRASH_YEAR", "CRASH_HOUR"]].head()


see number of crashes per year

In [ ]:
df_crashes["CRASH_YEAR"] = pd.to_datetime(df_crashes["CRASH DATE"], errors="coerce").dt.year

px.histogram(
    df_crashes,
    x="CRASH_YEAR",
    title="Number of Crashes per Year"
)


In [ ]:
df_borough = (
    df_crashes["BOROUGH"]
    .value_counts()
    .reset_index()
)

df_borough.columns = ["BOROUGH", "COUNT"]

fig = px.bar(
    df_borough,
    x="BOROUGH",
    y="COUNT",
    title="Crash Count per Borough"
)
fig.show()


crashes by vehicle type

In [ ]:
df_veh = (
    df_crashes["VEHICLE TYPE CODE 1"]
    .value_counts()
    .head(10)
    .reset_index()
)

df_veh.columns = ["VEHICLE_TYPE", "COUNT"]

fig = px.bar(
    df_veh,
    x="VEHICLE_TYPE",
    y="COUNT",
    title="Top 10 Vehicle Types in Crashes"
)
fig.show()


# correleation heatmap for numeric feilds

In [ ]:
num_df = df_crashes.select_dtypes(include=[np.number])

plt.figure(figsize=(12,10))
sns.heatmap(num_df.corr(), annot=False, cmap="coolwarm")
plt.title("Correlation Heatmap for Crash Numeric Variables")
plt.show()


pedestrians, cyclists crashjcount

In [ ]:
ped = df_crashes["NUMBER OF PEDESTRIANS INJURED"].sum()
cyc = df_crashes["NUMBER OF CYCLIST INJURED"].sum()
mot = df_crashes["NUMBER OF MOTORIST INJURED"].sum()

sns.barplot(x=["Pedestrians", "Cyclists", "Motorists"], y=[ped, cyc, mot])
plt.title("Total Injuries by Road User Type")
plt.show()


# Standardize strings and numeric casts and replacing NaN with 0

In [ ]:
def standardize_crashes(df):

    # Borough: strip + title case
    if "BOROUGH" in df.columns:
        df["BOROUGH"] = df["BOROUGH"].astype(str).str.strip().replace("nan", np.nan)
        df["BOROUGH"] = df["BOROUGH"].where(df["BOROUGH"].isna(), df["BOROUGH"].str.title())

    numeric_cols = [
        "NUMBER OF PERSONS INJURED",
        "NUMBER OF PERSONS KILLED",
        "NUMBER OF PEDESTRIANS INJURED",
        "NUMBER OF PEDESTRIANS KILLED",
        "NUMBER OF CYCLIST INJURED",
        "NUMBER OF CYCLIST KILLED",
        "NUMBER OF MOTORIST INJURED",
        "NUMBER OF MOTORIST KILLED"
    ]

    for c in numeric_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int)

    return df, numeric_cols   # return numeric_cols so it exists outside


df_crashes, numeric_cols = standardize_crashes(df_crashes)

df_crashes[numeric_cols].head()


In [ ]:
nyc_bbox = {
    "lat_min": 40.4774, "lat_max": 40.9176,
    "lon_min": -74.2591, "lon_max": -73.7004
}

# bashoof where is valid in NYC or not valid with a flag
def flag_latlon(df, lat_col="LATITUDE", lon_col="LONGITUDE"):
    df[lat_col] = pd.to_numeric(df.get(lat_col), errors="coerce")
    df[lon_col] = pd.to_numeric(df.get(lon_col), errors="coerce")
    df["latlon_valid"] = (
        df[lat_col].between(nyc_bbox["lat_min"], nyc_bbox["lat_max"]) &
        df[lon_col].between(nyc_bbox["lon_min"], nyc_bbox["lon_max"])
    )
    return df

df_crashes = flag_latlon(df_crashes)
df_crashes["latlon_valid"].value_counts(dropna=False)


checks whether each crash record has valid latitude and longitude coordinates inside New York City’s geographic boundaries.


In [ ]:
df_nyc = df_crashes[df_crashes["latlon_valid"] == True]
px.scatter(
    df_nyc.sample(30000),   # smaller sample = clearer plot
    x="LONGITUDE",
    y="LATITUDE",
    opacity=0.4,
    title="Geographic Distribution of Crash Coordinates (Cleaned NYC Only)"
)


# Injury severity distributions

In [ ]:
inj_cols = [
    "NUMBER OF PERSONS INJURED",
    "NUMBER OF PERSONS KILLED",
    "NUMBER OF PEDESTRIANS INJURED",
    "NUMBER OF PEDESTRIANS KILLED",
]

df_crashes[inj_cols].hist(bins=20, figsize=(12,8))
plt.suptitle("Injury and Fatality Distributions")


# IQR outlier detection on NUMBER OF PERSONS INJURED

In [ ]:
col = "NUMBER OF PERSONS INJURED"

q1 = df_crashes[col].quantile(0.25)
q3 = df_crashes[col].quantile(0.75)
iqr = q3 - q1

lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr

print(f"{col} IQR bounds: {lower} - {upper}")

outliers = df_crashes[(df_crashes[col] < lower) | (df_crashes[col] > upper)]
outliers.shape


# removing exact duplicates and rows with nothing.

In [ ]:
before = len(df_crashes)
df_crashes = df_crashes.drop_duplicates()
after = len(df_crashes)
print("Dropped exact duplicates:", before-after)

mask_missing_core = df_crashes["CRASH_DATETIME"].isna() & df_crashes["BOROUGH"].isna() & df_crashes["LATITUDE"].isna()
print("Rows missing core loc/time info:", mask_missing_core.sum())
df_crashes = df_crashes[~mask_missing_core].copy()


# Save intermediate cleaned crashes

In [ ]:
df_crashes_preint = df_crashes.copy()
df_crashes_preint.to_csv("crashes_pre_integration.csv", index=False)
print("Saved crashes_pre_integration.csv")


In [ ]:
df_persons.head(3).T
df_persons.info()


# See if collision ID actually exists

In [ ]:
if "COLLISION_ID" in df_persons.columns:
    print("Person rows per COLLISION_ID (sample):")
    print(df_persons.groupby("COLLISION_ID").size().sort_values(ascending=False).head())
else:
    print("No COLLISION_ID in persons dataset; need to find equivalent join key.")


crashes by hours of the day

In [ ]:
df_crashes["CRASH_TIME_HOUR"] = pd.to_datetime(df_crashes["CRASH TIME"], errors="coerce").dt.hour

px.histogram(
    df_crashes,
    x="CRASH_TIME_HOUR",
    nbins=24,
    title="Crashes Distribution by Hour of Day"
)


# Standardizing persons

In [ ]:
# Cell 13: Standardize persons
def standardize_persons(df):
    # trim strings
    for c in df.select_dtypes(include=["object"]).columns:
        df[c] = df[c].astype(str).str.strip().replace("nan", np.nan)
    # example numeric cast
    num_cols = ["AGE", "NUMBER_OF_PERSONS_INJURED", "NUMBER_OF_PERSONS_KILLED"]
    for c in num_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

df_persons = standardize_persons(df_persons)
df_persons.head(3)


Standerdizing column *names* to name them all in the same way


In [ ]:
df_crashes.columns = df_crashes.columns.str.replace(" ", "_").str.upper()

df_persons.columns = df_persons.columns.str.replace(" ", "_").str.upper()

In [ ]:
print(df_crashes.columns.tolist())
print(df_persons.columns.tolist())


# SAFE PERSON-LEVEL AGGREGATION

In [ ]:
df_persons["INJURED_FLAG"] = df_persons["PERSON_INJURY"].astype(str).str.upper().str.contains("INJUR").astype(int)
df_persons["KILLED_FLAG"]  = df_persons["PERSON_INJURY"].astype(str).str.upper().str.contains("KILL").astype(int)

persons_agg = df_persons.groupby("COLLISION_ID").agg(
    persons_count=("PERSON_ID", "count"),
    persons_injured=("INJURED_FLAG", "sum"),
    persons_killed=("KILLED_FLAG", "sum"),
    ped_involved=("PERSON_TYPE", lambda x: (x.str.upper().str.contains("PED")).any()),
    cyc_involved=("PERSON_TYPE", lambda x: (x.str.upper().str.contains("CYC")).any())
).reset_index()

persons_agg.to_csv("persons_aggregated_by_collision.csv", index=False)

print("Saved persons_aggregated_by_collision.csv", persons_agg.shape)


# The merge combines crash-level data + person-level aggregates

In [ ]:
df_join = df_crashes.merge(persons_agg, on="COLLISION_ID", how="left")

print("df_join shape:", df_join.shape)
df_join.head()


# Feature Engineering

In [ ]:
df_join["CRASH_DATETIME"] = pd.to_datetime(
    df_join["CRASH_DATE"].astype(str) + " " + df_join["CRASH_TIME"].astype(str),
    errors="coerce"
)

df_join["YEAR"] = df_join["CRASH_DATETIME"].dt.year
df_join["MONTH"] = df_join["CRASH_DATETIME"].dt.month
df_join["WEEKDAY"] = df_join["CRASH_DATETIME"].dt.day_name()
df_join["HOUR"] = df_join["CRASH_DATETIME"].dt.hour

df_join["SEVERE_FLAG"] = ((df_join["persons_injured"] > 0) |
                          (df_join["persons_killed"] > 0)).astype(int)



# Build final cleaned merged crash dataset

In [ ]:
crash_cols = [
    "COLLISION_ID",
    "CRASH_DATETIME", "YEAR", "MONTH", "WEEKDAY", "HOUR",
    "BOROUGH", "ZIP_CODE",
    "LATITUDE", "LONGITUDE",
    "ON_STREET_NAME", "CROSS_STREET_NAME",
    "NUMBER_OF_PERSONS_INJURED", "NUMBER_OF_PERSONS_KILLED",
    "NUMBER_OF_PEDESTRIANS_INJURED", "NUMBER_OF_PEDESTRIANS_KILLED",
    "NUMBER_OF_CYCLIST_INJURED", "NUMBER_OF_CYCLIST_KILLED",
    "NUMBER_OF_MOTORIST_INJURED", "NUMBER_OF_MOTORIST_KILLED",
    "CONTRIBUTING_FACTOR_VEHICLE_1",
    "VEHICLE_TYPE_CODE_1",
    "persons_count", "persons_injured", "persons_killed",
    "ped_involved", "cyc_involved",
    "SEVERE_FLAG"
]

crash_cols = [c for c in crash_cols if c in df_join.columns]

df_final = df_join[crash_cols].copy()
df_final.to_csv("merged_cleaned2.csv", index=False)

print("Saved merged_cleaned.csv with shape:", df_final.shape)



In [ ]:
df_final = pd.read_csv("/content/merged_cleaned2.csv")


# Start Visualizations

In [ ]:
borough_year = (
    df_final.groupby(["BOROUGH", "YEAR"])["COLLISION_ID"]
    .nunique()
    .reset_index()
)

fig = px.line(
    borough_year,
    x="YEAR",
    y="COLLISION_ID",
    color="BOROUGH",
    title="Crashes by Borough per Year"
)
fig.show()


In [ ]:
ped = df_final[df_final["ped_involved"] == True]

ped_year = (
    ped.groupby(["BOROUGH", "YEAR"])["COLLISION_ID"]
    .nunique()
    .reset_index()
)

fig = px.line(
    ped_year,
    x="YEAR",
    y="COLLISION_ID",
    color="BOROUGH",
    title="Pedestrian-Involved Crashes Across Years by Borough"
)
fig.show()


In [ ]:
fatal = df_final[df_final["persons_killed"] > 0]

factor_counts = (
    fatal["CONTRIBUTING_FACTOR_VEHICLE_1"]
    .value_counts()
    .head(15)
    .reset_index()
)

factor_counts.columns = ["Factor", "Fatal Crashes"]

fig = px.bar(
    factor_counts,
    x="Factor",
    y="Fatal Crashes",
    title="Top Contributing Factors in Fatal Crashes"
)
fig.show()


In [ ]:
veh_sev = (
    df_final.groupby("VEHICLE_TYPE_CODE_1")["SEVERE_FLAG"]
    .mean()
    .reset_index()
    .sort_values("SEVERE_FLAG", ascending=False)
    .head(20)
)

fig = px.bar(
    veh_sev,
    x="VEHICLE_TYPE_CODE_1",
    y="SEVERE_FLAG",
    title="Vehicle Types with Highest Severe Crash Rates"
)
fig.show()


In [ ]:
zip_hotspots = (
    df_final.groupby("ZIP_CODE")["SEVERE_FLAG"]
    .sum()
    .reset_index()
    .sort_values("SEVERE_FLAG", ascending=False)
    .head(20)
)

fig = px.bar(
    zip_hotspots,
    x="ZIP_CODE",
    y="SEVERE_FLAG",
    title="Top 20 ZIP Codes with Most Severe Crashes"
)
fig.show()


In [ ]:
fatal_hour = (
    df_final[df_final["persons_killed"] > 0]
    .groupby("HOUR")["COLLISION_ID"]
    .nunique()
    .reset_index()
)

fig = px.line(
    fatal_hour,
    x="HOUR",
    y="COLLISION_ID",
    title="Fatal Crashes by Hour of Day"
)
fig.show()


In [ ]:
monthly_severe = (
    df_final.groupby("MONTH")["SEVERE_FLAG"]
    .sum()
    .reset_index()
)

fig = px.line(
    monthly_severe,
    x="MONTH",
    y="SEVERE_FLAG",
    title="Seasonal Trend of Severe Crashes"
)
fig.show()


In [ ]:
df_final["INTERSECTION"] = (
    df_final["ON_STREET_NAME"].astype(str)
    + " / "
    + df_final["CROSS_STREET_NAME"].astype(str)
)

top_intersections = (
    df_final.groupby("INTERSECTION")["COLLISION_ID"]
    .nunique()
    .reset_index()
    .sort_values("COLLISION_ID", ascending=False)
    .head(20)
)

fig = px.bar(
    top_intersections,
    x="INTERSECTION",
    y="COLLISION_ID",
    title="Top 20 Most Dangerous Intersections"
)
fig.show()


In [ ]:
# A simple proxy: multi-vehicle if more than one motorist injury or fatality
df_final["MULTI_VEHICLE"] = (
    df_final["NUMBER_OF_MOTORIST_INJURED"]
    + df_final["NUMBER_OF_MOTORIST_KILLED"]
) > 1

sev_mv = (
    df_final.groupby("MULTI_VEHICLE")["SEVERE_FLAG"]
    .mean()
    .reset_index()
)

fig = px.bar(
    sev_mv,
    x="MULTI_VEHICLE",
    y="SEVERE_FLAG",
    title="Severity Rate: Multi-Vehicle vs Single-Vehicle Crashes"
)
fig.show()


In [ ]:
df_people = df_persons.merge(
    df_crashes[["COLLISION_ID", "BOROUGH", "CRASH_DATE"]],
    on="COLLISION_ID",
    how="left"
)

age_sev = (
    df_people.groupby("PERSON_AGE")["INJURED_FLAG"]
    .mean()
    .reset_index()
)

fig = px.line(
    age_sev,
    x="PERSON_AGE",
    y="INJURED_FLAG",
    title="Injury Probability by Age"
)
fig.show()


In [ ]:
sex_sev = (
    df_people.groupby("PERSON_SEX")["INJURED_FLAG"]
    .mean()
    .reset_index()
)

fig = px.bar(
    sex_sev,
    x="PERSON_SEX",
    y="INJURED_FLAG",
    title="Injury Probability by Sex"
)
fig.show()


# Salma Hossam: pre-integration EDA and cleaing

# Dareen Marwan: pre-integration cleaning and merging

# Farah Sherif: post integration cleaning and visualizations

# Omar belal and Habiba Salama:  dash/plotly website and deployment